In [1]:
import pandas as pd
import numpy as np
import matplotlib
# matplotlib.use('Agg')
%matplotlib inline
matplotlib.use('module://ipykernel.pylab.backend_inline')
import matplotlib.pyplot as plt
import math

from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    

import statsmodels.api as sm
import copy
import os
import shutil

%matplotlib inline
matplotlib.use('module://ipykernel.pylab.backend_inline')
import matplotlib.pyplot as plt


In [2]:
n_samples = None # if none then all data points are taken, otherwise only the first n_samples, used for debugging


lasso_epsilon = 0.01
p_val = 0.05
age_limit = 0
gender = None
gender_column = "Sex"
iqr_coefficient = 1.5 # if None then no standrat removal of outliers

simple_predictors =["Age", "BMI"]
predictors = ["HighBP", "HighChol", "CholCheck", "BMI","Smoker", "Stroke", 
              "HeartDiseaseorAttack", "MentHlth", "PhysActivity", "DiffWalk", "Fruits", "Veggies", "HvyAlcoholConsump", 
             "AnyHealthcare", "NoDocbcCost", "Sex",  "Age", "Education", "Income"]

binarys = ["HighBP", "HighChol"]

# set your outcome variable here
outcome =  "Diabetes_012"


HSCD = 'HC1' # if None than the model builder will be heteroscedascisity unaware
remove_na = True # must be set to true for input data set with the NA values present

# set your wokring directory here, it containes input csv file and will contain output files (model)

home_directory = os.path.expanduser("~")
working_dir = f"{home_directory}/PRIME/example_data"

# source https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators
# source https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset?resource=download
# set locations of your input csv data set here
input_file = f"{working_dir}/diabetes_012_health_indicators_BRFSS2015.csv"




In [3]:

# read the input data set
df = pd.read_table(input_file, nrows=n_samples, sep=',')

# creating output directory within the working directory, the name of the output directory will tell how model is build
# the creation of the output name follows straightforward from the code frgament below

outcome_dir = outcome.replace(',', '_').replace('/', '_').replace(' ', '')
if "imputed" in input_file:
  output_dir = f"{working_dir}/imputed_{outcome_dir}"
else:
  df = df.dropna(axis="rows")
  print(f"cleaned check up db has {len(df)} rows")
  output_dir = f"{working_dir}/non_na_{outcome_dir}"
  


print(df.head())

if HSCD is not None:
  output_dir = f"{output_dir}_HSCD_{HSCD}"

if gender is None:
  output_dir = f"{output_dir}/"
else:
  output_dir = f"{output_dir}_gender_{gender}"
  df = df[df[gender_column].eq(gender)]
  df = df.drop(columns=[gender_column])
  predictors.remove(gender_column)

if not os.path.exists(output_dir):
  os.makedirs(output_dir)

print(output_dir)

# finished creating output directory
  
# we need to check for each predictor if  not all its values are zeros
for predictor in binarys:
    if df[predictor].eq(1).sum() == 0:
      df = df.drop(columns=[predictor])
      predictors.remove(predictor)
      print(f"dropped column because all zeroes: {predictor}")
    if df[predictor].eq(0).sum() == 0:
      df = df.drop(columns=[predictor])
      predictors.remove(predictor)
      print(f"dropped column because all ones: {predictor}")



cleaned check up db has 253680 rows
   Diabetes_012  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0           0.0     1.0       1.0        1.0  40.0     1.0     0.0   
1           0.0     0.0       0.0        0.0  25.0     1.0     0.0   
2           0.0     1.0       1.0        1.0  28.0     0.0     0.0   
3           0.0     1.0       0.0        1.0  27.0     0.0     0.0   
4           0.0     1.0       1.0        1.0  24.0     0.0     0.0   

   HeartDiseaseorAttack  PhysActivity  Fruits  ...  AnyHealthcare  \
0                   0.0           0.0     0.0  ...            1.0   
1                   0.0           1.0     0.0  ...            0.0   
2                   0.0           0.0     1.0  ...            1.0   
3                   0.0           1.0     1.0  ...            1.0   
4                   0.0           1.0     1.0  ...            1.0   

   NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex   Age  Education  \
0          0.0      5.0      18.0      15.0      

In [4]:
df = df[predictors+[outcome]]
if remove_na:
    df = df.dropna(axis="rows")
    print(f"cleaned check up db has {len(df)} rows")

cleaned check up db has 253680 rows


In [5]:

if (gender is not None) and (gender_column in simple_predictors):
    simple_predictors.remove(gender_column) 

In [6]:
global_predictors = predictors
for predictor in global_predictors:
    if predictor not in df.columns:
        print(f"predictor column  is missing {predictor}")

In [7]:
def model_builder(df_source, predictors, outcome_var, result_dir, hcd):
    
    if not os.path.exists(result_dir):
    # The directory does not exist, create it
        os.makedirs(result_dir)
        
    y = df_source[outcome_var]
    print(f"outcome: {y}")
    
    X = df_source[predictors]
        
  
    # getting basic statistics for all predictors and standartising on the flow
   
    df_mean_std = pd.DataFrame(columns = ["predictor", "mean", "standard_deviation", "min", "max"] )
    
    predictors = X.columns 
    for i in  range(len(predictors)):  
        clmn = predictors[i]
        mean_ = X[clmn].mean()
        std_ = X[clmn].std()
        df_mean_std.loc[i] = [clmn, mean_, std_, X[clmn].min(), X[clmn].max()]
        X[clmn] = (X[clmn] - mean_)/std_
        
   
    X_with_intercept = sm.add_constant(X) 
    if hcd is not None:
        model_1= sm.OLS(y, X_with_intercept).fit(cov_type=hcd) 
    else:
        model_1= sm.OLS(y, X_with_intercept).fit() 
    
    
    with open(f"{result_dir}/simle_ols_model_summary.txt", 'w') as model_1_file:
    # Convert the summary to a string and write it to the file
        model_1_file.write(model_1.summary().as_text())
    
    coefficients = model_1.params
    p_values = model_1.pvalues
    df_results = pd.DataFrame({'Coefficient': coefficients, 'P_value': p_values})
    
    df_results = df_results.reset_index()
    df_results = df_results.rename(columns={df_results.columns[0]: 'predictor'})
    df_results = pd.merge(df_results, df_mean_std, on = "predictor", how = 'outer')
    
    df_results.to_csv(f"{result_dir}/simple_ols_model_with_stats.csv", sep=',')
  
    return df_results, model_1

In [8]:
def lasso_model_builder(df_source, predictors, outcome_var, result_dir, hcd):
    
    if not os.path.exists(result_dir):
    # The directory does not exist, create it
        os.makedirs(result_dir)
        
    y = df_source[outcome_var]
    print(f"outcome: {y}")
    
    X = df_source[predictors]
   
    
    ############ LASSO ###### 
    
    scaler = StandardScaler()
    # Initialize LassoCV with 100-fold cross-validation
    lasso_cv = LassoCV(cv=100, random_state=42, max_iter=10000)
    pipeline = make_pipeline(scaler, lasso_cv)
    pipeline.fit(X, y)
        
    lasso_cv_fitted = pipeline.named_steps['lassocv']
    
    summary_text = f"Best alpha (regularization strength): {lasso_cv_fitted.alpha_}\n"
    summary_text += f"Intercept {lasso_cv_fitted.intercept_}\nCoefficients:\n"
    summary_text += f"{lasso_cv_fitted.coef_}\n"
    summary_text += f"MSE path {lasso_cv_fitted.mse_path_}\n"
        
    r2_score = pipeline.score(X, y)
    summary_text += f"R^2 score: {r2_score}"
   
    with open(f"{result_dir}/intermediate_results_lasso_technical_summary.txt", 'w') as lasso_file:
        lasso_file.write(summary_text)
    
    # preparing the list of predictors with non zero coefficients (large enough)
    if abs(lasso_cv_fitted.intercept_) >=  lasso_epsilon:
        selected_features = ["const"]  
    else:  
        selected_features = []
        
    for i in range(len(predictors)):
        if abs(lasso_cv_fitted.coef_[i])>= lasso_epsilon:
            selected_features = selected_features + [predictors[i]]
   
    
    # getting coefficients and basic statistics for all predictors, incl const
   
    df_mean_std = pd.DataFrame(columns = ["predictor", "Coefficient", "mean", "standard_deviation", "min", "max"] )
    df_mean_std.loc[0] = ["const", lasso_cv_fitted.intercept_, lasso_cv_fitted.intercept_, 0, lasso_cv_fitted.intercept_, lasso_cv_fitted.intercept_]
    X_scaled = df_source[predictors]
    for i in  range(len(predictors)):  
        clmn = predictors[i]
        
        mean_ = X_scaled[clmn].mean()
        std_ = X_scaled[clmn].std()
        df_mean_std.loc[i+1] = [clmn, lasso_cv_fitted.coef_[i], mean_, std_, X_scaled[clmn].min(), X_scaled[clmn].max()]
        X_scaled[clmn] =  (X_scaled[clmn] - mean_)/std_
        
    ## getting p-values for all predictors via ols model
   
    X_scaled_with_intercept = sm.add_constant(X_scaled) 
    if hcd is not None:
        model_1 = sm.OLS(y, X_scaled_with_intercept).fit(cov_type=hcd) 
    else:
        model_1 = sm.OLS(y, X_scaled_with_intercept).fit() 
    
    
    # with open(f"{result_dir}/ols_model_summary.txt", 'w') as model_1_file:
    # Convert the summary to a string and write it to the file
        # model_1_file.write(model_1.summary().as_text())
    
    coefficients = model_1.params
    p_values = model_1.pvalues
    df_results = pd.DataFrame({'OLS_Coefficient': coefficients, 'P_value': p_values})
    
    df_results = df_results.reset_index()
    df_results = df_results.rename(columns={df_results.columns[0]: 'predictor'})
    df_results = pd.merge(df_results, df_mean_std, on = "predictor", how = 'outer')
    
    df_results.to_csv(f"{result_dir}/intermediate_results_lasso_readable.csv", sep=',')
    
    # creating OLS model based on selected predictors 
    if 'const' in selected_features:
        selected_features.remove('const')
        
    X_selected = X_scaled[selected_features]
    X_selected_with_intercept = sm.add_constant(X_selected) 
    if hcd is not None:
        model_2= sm.OLS(y, X_selected_with_intercept).fit(cov_type=hcd) 
    else:
        model_2= sm.OLS(y, X_selected_with_intercept).fit() 
        
    with open(f"{result_dir}/final_aftre_lasso_OSL_model_summary.txt", 'w') as model_2_file:
    # Convert the summary to a string and write it to the file
        model_2_file.write(model_2.summary().as_text())
        
    
    coefficients_2 = model_2.params
    p_values_2 = model_2.pvalues
    df_selected_results = pd.DataFrame({'OLS_Coefficient': coefficients_2, 'P_value': p_values_2})
    
    df_selected_results = df_selected_results.reset_index()
    df_selected_results = df_selected_results.rename(columns={df_selected_results.columns[0]: 'predictor'})
    df_selected_results = pd.merge(df_selected_results, df_mean_std, on = "predictor", how = 'outer')
    
    df_selected_results = df_selected_results[df_selected_results['predictor'].isin(selected_features + ['const'])]
    df_selected_results.to_csv(f"{result_dir}/final_after_lasso_OSL_model_with_stats.csv", sep=',')
    
    return df_selected_results, model_2

In [9]:
def show_dependencies(df_, model_df, predictors_to_show, dir):
    
    if model_df is None:
        return
    
    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.makedirs(dir)
    
    print("we are building graphs")
    
    for predictor in predictors_to_show:
        
        
        print(predictor)
       
        y = np.zeros(df_.shape[0])
        # 
        # aggregate all the terms containing pedictor 
        for feat in model_df['predictor']:
            if not predictor in feat:
                continue
            if not feat in df_.columns:
                continue
            print(feat)
            
            # prepare the feature
            match feat:
                case _ if feat == predictor:
                    fun_pred = df_[predictor]
                case _ if feat == f"log_{predictor}":
                    fun_pred = np.log1p(df_[predictor])
                case _ if feat == f"quadr_{predictor}":
                    fun_pred = np.square(df_[predictor])
                case _ if feat == f"quadr_log_{predictor}":
                    fun_pred = np.square(np.log1p(df_[predictor]))
                case _:
                    fun_pred = None
                    print(f"error, non existing predictor {feat}")
            # standartise the feature
            fun_pred = (fun_pred - fun_pred.mean())/fun_pred.std()
            
            
            
            # find the coefficient
            row_feat = model_df[model_df['predictor'] == feat].iloc[0]
            coeff =  row_feat["Coefficient"]
            
            y = y + coeff *fun_pred
        
        # x_pred = (df_[predictor]    - df_[predictor].mean()) /  df_[predictor].std()
        # x = np.linspace(min(x_pred), max(x_pred), 100)  
          
        are_all_zeros = np.all(y == 0)        
        if not are_all_zeros:
            x = df_[predictor]
        
            plt.scatter(x, y)
            plt.xlabel(predictor)
            plt.ylabel('outcome')
            plt.legend()
            plt.grid(True)

            # Save the plot as a PNG file
            plt.savefig(f"{dir}/{predictor}.png", format='png', dpi=300)
            plt.show()

In [10]:
if lasso_epsilon is not None :
        
    model_table, model = lasso_model_builder(df, global_predictors, outcome, f"{output_dir}",  HSCD)
    model_table_simple, model_simple = model_builder(df, simple_predictors, outcome, f"{output_dir}",  HSCD)
            
else:

    X = df[global_predictors]
    y = df[outcome]
    assert(len(X) == len(y))
   
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=1)
    
    assert(len(X_train) == len(y_train))
    assert(len(X_val) == len(y_val))
    
    '''
    X_train, X_v0, y_train, y_v0 = train_test_split(X, y, test_size=0.4, random_state=1)
    
    assert(len(X_train) == len(y_train))
    assert(len(X_v0) == len(y_v0))
    
    
    X_val, X_test, y_val, y_test = train_test_split(X_v0, y_v0, test_size=0.5, random_state=1)
    assert(len(X_val) == len(y_val))
    assert(len(X_test) == len(y_test))
    '''
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    # main model    
    model = LinearRegression()
    model.fit(X_train_scaled, y_train)
    # show_dependencies(df, model_table, predictors_to_plot, f"{output_dir}/figures")
  
    
    y_pred_val = model.predict(X_val_scaled)
   
    mae = mean_absolute_error(y_val, y_pred_val)
    mse = mean_squared_error(y_val, y_pred_val)
    rmse = mean_squared_error(y_val, y_pred_val, squared=False)  # RMSE is the square root of MSE
    r2 = r2_score(y_val, y_pred_val)
    
    validation_summary = f"main model vaidation mean_absolute_error  { mae}\n"
    validation_summary += f"main model vaidation root mean_squared_error  {rmse}\n"
    validation_summary += f"main model vaidation R-squared (the coefficient of determination) {r2}\n"
    
    
    # simple model      
    X_train_simple = X_train[simple_predictors]
    X_val_simple = X_val[simple_predictors]
    scaler_simple = StandardScaler()
    X_train_simple_scaled = scaler_simple.fit_transform(X_train_simple)
    X_val_simple_scaled = scaler_simple.transform(X_val_simple)
    
    model_simple = LinearRegression()
    model_simple.fit(X_train_simple_scaled, y_train)
       
    
    y_simple_pred_val = model_simple.predict(X_val_simple_scaled)
    mae_simple = mean_absolute_error(y_val, y_simple_pred_val)
    mse_simple = mean_squared_error(y_val, y_simple_pred_val)
    rmse_simple = mean_squared_error(y_val, y_simple_pred_val, squared=False)  # RMSE is the square root of MSE
    r2_simple = r2_score(y_val, y_simple_pred_val)
    
    validation_summary += f"simple model vaidation mean_absolute_error  { mae_simple}\n"
    validation_summary += f"simple model vaidation root mean_squared_error  {rmse_simple}\n"
    validation_summary += f"simple model vaidation model R-squared (the coefficient of determination) {r2_simple}\n"
    
    print(validation_summary)
    with open(f"{output_dir}/validation_summary_wo_lasso.txt", 'w') as sum_file:
        sum_file.write(validation_summary)
    
            
            

outcome: 0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
253675    0.0
253676    2.0
253677    0.0
253678    0.0
253679    2.0
Name: Diabetes_012, Length: 253680, dtype: float64


/var/folders/87/pb6c775x4ql_2tjlhmgjnh1w0000gn/T/ipykernel_32721/2607438535.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_scaled[clmn] =  (X_scaled[clmn] - mean_)/std_
/var/folders/87/pb6c775x4ql_2tjlhmgjnh1w0000gn/T/ipykernel_32721/2607438535.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_scaled[clmn] =  (X_scaled[clmn] - mean_)/std_
/var/folders/87/pb6c775x4ql_2tjlhmgjnh1w0000gn/T/ipykernel_32721/2607438535.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

outcome: 0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
253675    0.0
253676    2.0
253677    0.0
253678    0.0
253679    2.0
Name: Diabetes_012, Length: 253680, dtype: float64


/var/folders/87/pb6c775x4ql_2tjlhmgjnh1w0000gn/T/ipykernel_32721/26120744.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[clmn] = (X[clmn] - mean_)/std_
/var/folders/87/pb6c775x4ql_2tjlhmgjnh1w0000gn/T/ipykernel_32721/26120744.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[clmn] = (X[clmn] - mean_)/std_
